In [13]:
import numpy #Matirx algebra
import urllib #Read data from web
import scipy.optimize #Optimization
import random

In [14]:
def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)

In [15]:
def parseDataFromFile(fname):
    for l in open(fname):
        yield eval(l)

In [16]:
print "Reading data..."
#data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
data = list(parseDataFromFile("/Users/Vernonld/Desktop/beer_50000.json"))
print "done"

Reading data...
done


In [29]:
for d in data:
    if d["beer/brewerId"] == "1075":
        print d
        break

{'review/appearance': 4.0, 'review/timeUnix': 1293735206, 'beer/style': 'American Double / Imperial IPA', 'review/palate': 4.0, 'review/taste': 4.5, 'beer/name': 'Cauldron DIPA', 'review/text': "According to the website, the style for the Caldera Cauldron changes every year. The current release is a DIPA, which frankly is the only cauldron I'm familiar with (it was an IPA/DIPA the last time I ordered a cauldron at the horsebrass several years back). In any event... at the Horse Brass yesterday.\t\tThe beer pours an orange copper color with good head retention and lacing. The nose is all hoppy IPA goodness, showcasing a huge aroma of dry citrus, pine and sandlewood. The flavor profile replicates the nose pretty closely in this West Coast all the way DIPA. This DIPA is not for the faint of heart and is a bit much even for a hophead like myslf. The finish is quite dry and hoppy, and there's barely enough sweet malt to balance and hold up the avalanche of hoppy bitterness in this beer. Mou

In [5]:
#Preprocessing Dataset in order to eliminate the unreasonable data with inaccurate age
#data = [d for d in data if 'user/ageInSeconds' in d and d['user/ageInSeconds'] < 60*60*24*365*80]

In [6]:
def feature(datum):
    feat = [1]
    return feat


# Question 1
How many reviews are there for each style of beer in the dataset (‘beer/style’)? What is the average value of ‘review/taste’ for reviews from each style? (1 mark)

In [7]:
beer_dic = {}
beer_review = {}
beer_avg = {}
beer_type = set([d['beer/style'] for d in data])
for beer in beer_type:
    if beer not in beer_dic:
        beer_dic[beer] = 0
    if beer not in beer_review:
        beer_review[beer] = 0    
    for d in data:
        if d['beer/style'] == beer:
            beer_dic[beer] += 1
            beer_review[beer] += d['review/taste']

for beer in beer_dic:
    beer_avg[beer] = beer_review[beer] / beer_dic[beer]
    

In [8]:
print "{:<40} {:<15}".format('beer/style','Number')
for k, v in beer_dic.iteritems():
    print "{:<40} {:<15} ".format(k, v)

beer/style                               Number         
Tripel                                   257             
American Black Ale                       138             
Fruit / Vegetable Beer                   1355            
English Brown Ale                        495             
American Pale Ale (APA)                  2288            
American Amber / Red Lager               42              
Chile Beer                               11              
Cream Ale                                69              
Dubbel                                   165             
English Dark Mild Ale                    21              
Belgian Strong Pale Ale                  632             
Weizenbock                               13              
Milk / Sweet Stout                       69              
Dunkelweizen                             61              
Belgian Strong Dark Ale                  146             
Foreign / Export Stout                   55              
American Porter

In [9]:
print "{:<40} {:<15}".format('beer/style','review/taste/average')
for k, v in beer_avg.iteritems():
    print "{:<40} {:<15} ".format(k, v)

beer/style                               review/taste/average
Tripel                                   3.78404669261   
American Black Ale                       3.8731884058    
Fruit / Vegetable Beer                   3.60774907749   
English Brown Ale                        3.72828282828   
American Pale Ale (APA)                  3.64969405594   
American Amber / Red Lager               3.69047619048   
Chile Beer                               3.95454545455   
Cream Ale                                3.02898550725   
Dubbel                                   3.73636363636   
English Dark Mild Ale                    3.78571428571   
Belgian Strong Pale Ale                  4.05617088608   
Weizenbock                               3.38461538462   
Milk / Sweet Stout                       3.78260869565   
Dunkelweizen                             3.49180327869   
Belgian Strong Dark Ale                  3.69520547945   
Foreign / Export Stout                   3.25454545455   
Low Alcoho

# Question 2
Train a simple predictor with a single binary feature indicating whether a beer is an ‘American IPA’: review/taste ≃ θ0 + θ1 × [beer is an American IPA]
Report the values of θ0 and θ1. Briefly describe your interpretation of these values, i.e., what do θ0 and θ1 represent (1 mark)?

* _θ0 is the average review scores of beers other than AmericanIPA, θ1 is the average score that AmericanIPA is higher than others_ *

In [10]:
### TASK2 2
def feature(datum):
    feat = [1]
    if datum['beer/style'] == 'American IPA':
        feat.append(1)
    else:
        feat.append(0)
    return feat

data3 = [d for d in data]
x = [feature(d) for d in data3]
y = [d['review/taste'] for d in data3]
theta,residuals,rank,s = numpy.linalg.lstsq(x, y)
print theta

[ 3.91520474  0.08564622]


# Question 3
Split the data into two equal fractions – the first half for training, the second half for testing (based on the order they appear in the file). Train the same model as above on the training set only. What is the model’s MSE on the training and on the test set (1 mark)?

In [11]:
### TASK3
# X = numpy.matrix(X)
# y = numpy.matrix(y)
# print numpy.linalg.inv(X.T * X) * X.T * y.T
def mse(theta, x, y):
    theta = numpy.matrix(theta)
    x     = numpy.matrix(x)
    y     = numpy.matrix(y)
    mse = (numpy.square(y.T - (x * theta.T))).mean(axis=None)
    return mse

    
data3 = [d for d in data]
x = [feature(d) for d in data3]
y = [d['review/taste'] for d in data3]
theta,residuals,rank,s = numpy.linalg.lstsq(x[:25000], y[:25000])
mse_train = residuals/25000
mse_test  = mse(theta, x[25000:], y[25000:])
print mse_test, mse_train

        

0.468410050967 [ 0.55810729]


# Question 4
Extend the model above so that it incorporates binary features for every style of beer with ≥ 50 reviews. Report the values of θ that you obtain, and the model’s MSE on the training and on the test set (1 mark).

In [12]:
###Task4
def feature(datum):
    feat = [1]
    if datum['beer/style'] == 'American IPA':
        feat.append(1)
    else:
        feat.append(0)
    for beer in beer_dic:
        if beer_dic[beer] > 50:
            if datum['beer/style'] == beer:
                feat.append(1)
            else :
                feat.append(0)
    return feat

data4 = [d for d in data]
x = [feature(d) for d in data4]
y = [d['review/taste'] for d in data4]
theta,residuals,rank,s = numpy.linalg.lstsq(x[:25000], y[:25000])
mse_train = residuals/25000
mse_test  = mse(theta, x[25000:], y[25000:])
print theta
print mse_train, mse_test
    

[  3.60681818e+00  -2.07766643e+11   6.00738041e-02   2.02974651e-01
   1.13172196e-01   3.88227610e-02   2.70859595e-02  -6.62633274e-01
   1.41622086e-01   4.68242645e-01   7.10627325e-01   2.56204665e-01
   1.24126434e-01  -3.13223499e-01   3.30821991e-01  -9.20299530e-01
   2.67143250e-01  -5.21030650e-01  -9.81678516e-02   1.63909435e-01
   2.32016206e-01  -2.23141193e-01   1.49784446e-01  -4.14722919e-01
  -7.43757725e-01   2.18113661e-02   2.76198268e-01   9.82036591e-02
   2.09256411e-01  -1.84951305e-01  -1.32707596e-01  -2.53467560e-02
  -4.73805785e-01  -2.38751829e-01   4.56432343e-01   6.48686856e-01
  -2.68540859e-01   4.44286048e-01  -1.77109241e-01  -8.23332787e-01
  -3.18038940e-01  -3.96466821e-01  -7.40783215e-01   1.11326933e-01
   2.07766643e+11   7.64946461e-01  -5.77926636e-03  -1.04284763e-01
  -5.80929339e-01   3.42063427e-01   2.67836571e-01   3.35987091e-01
  -6.40163660e-01   5.87407589e-01   3.92505907e-01  -1.23380470e+00
   6.95693970e-01   3.80527377e-01